## Import all packages and functions needed

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## Decision tree class

In [ ]:
class dtree:
	""" A basic Decision Tree"""

	def __init__(self):
		""" Constructor """

	def read_data(self, filename):
		fid = open(filename, "r")
		data = []
		d = []
		for line in fid.readlines():
			d.append(line.strip())
		for d1 in d:
			data.append(d1.split(","))
		fid.close()

		self.featureNames = data[0]
		self.featureNames = self.featureNames[:-1]
		data = data[1:]
		self.classes = []
		for d in range(len(data)):
			self.classes.append(data[d][-1])
			data[d] = data[d][:-1]

		return data, self.classes, self.featureNames

	def classify(self, tree, datapoint):

		if type(tree) == type("string"):
			# Have reached a leaf
			return tree
		else:
			a = list(tree.keys())[0]
			for i in range(len(self.featureNames)):
				if self.featureNames[i] == a:
					break
			try:
				t = tree[a][datapoint[i]]
				return self.classify(t, datapoint)
			except:
				return None

	def classifyAll(self, tree, data):
		results = []
		for i in range(len(data)):
			results.append(self.classify(tree, data[i]))
		return results

	def make_tree(self,
				  data,
				  classes,
				  featureNames,
				  maxlevel=-1,
				  level=0,
				  forest=0):
		""" The main function, which recursively constructs the tree"""

		nData = len(data)
		nFeatures = len(data[0])

		try:
			self.featureNames
		except:
			self.featureNames = featureNames

		# List the possible classes
		newClasses = []
		for aclass in classes:
			if newClasses.count(aclass) == 0:
				newClasses.append(aclass)

		# Compute the default class (and total entropy)
		frequency = np.zeros(len(newClasses))

		totalEntropy = 0
		totalGini = 0
		index = 0
		for aclass in newClasses:
			frequency[index] = classes.count(aclass)
			totalEntropy += self.calc_entropy(float(frequency[index]) / nData)
			totalGini += (float(frequency[index]) / nData)**2

			index += 1

		totalGini = 1 - totalGini
		default = classes[np.argmax(frequency)]

		if nData == 0 or nFeatures == 0 or (maxlevel >= 0
											and level > maxlevel):
			# Have reached an empty branch
			return default
		elif classes.count(classes[0]) == nData:
			# Only 1 class remains
			return classes[0]
		else:

			# Choose which feature is best
			gain = np.zeros(nFeatures)
			ggain = np.zeros(nFeatures)
			featureSet = list(range(nFeatures))
			if forest != 0:
				np.random.shuffle(featureSet)
				featureSet = featureSet[0:forest]
			for feature in featureSet:
				g, gg = self.calc_info_gain(data, classes, feature)
				gain[feature] = totalEntropy - g
				ggain[feature] = totalGini - gg

			bestFeature = np.argmax(gain)
			tree = {featureNames[bestFeature]: {}}

			# List the values that bestFeature can take
			values = []
			for datapoint in data:
				if datapoint[feature] not in values:
					values.append(datapoint[bestFeature])

			for value in values:
				# Find the datapoints with each feature value
				newData = []
				newClasses = []
				index = 0
				for datapoint in data:
					if datapoint[bestFeature] == value:
						if bestFeature == 0:
							newdatapoint = datapoint[1:]
							newNames = featureNames[1:]
						elif bestFeature == nFeatures:
							newdatapoint = datapoint[:-1]
							newNames = featureNames[:-1]
						else:
							newdatapoint = datapoint[:bestFeature]
							# newdatapoint.append(datapoint[bestFeature+1:])
							newdatapoint = np.append(
								newdatapoint, datapoint[bestFeature + 1:])
							newNames = featureNames[:bestFeature]
							# newNames.append(featureNames[bestFeature+1:])
							newNames = np.append(
								newNames, featureNames[bestFeature + 1:])
						newData.append(newdatapoint)
						newClasses.append(classes[index])
					index += 1

				# Now recurse to the next level
				subtree = self.make_tree(newData, newClasses, newNames,
										 maxlevel, level + 1, forest)

				# And on returning, add the subtree on to the tree
				tree[featureNames[bestFeature]][value] = subtree

			return tree

	def printTree(self, tree, name):
		if type(tree) == dict:
			print(name, tree.keys()[0])
			for item in tree.values()[0].keys():
				print(name, item)
				self.printTree(tree.values()[0][item], name + "\t")
		else:
			print(name, "\t->\t", tree)

	def calc_entropy(self, p):
		if p != 0:
			return -p * np.log2(p)
		else:
			return 0

	def calc_info_gain(self, data, classes, feature):

		# Calculates the information gain based on both entropy and the Gini impurity
		gain = 0
		ggain = 0
		nData = len(data)

		# List the values that feature can take

		values = []
		for datapoint in data:
			if datapoint[feature] not in values:
				values.append(datapoint[feature])

		featureCounts = np.zeros(len(values))
		entropy = np.zeros(len(values))
		gini = np.zeros(len(values))
		valueIndex = 0
		# Find where those values appear in data[feature] and the corresponding class
		for value in values:
			dataIndex = 0
			newClasses = []
			for datapoint in data:
				if datapoint[feature] == value:
					featureCounts[valueIndex] += 1
					newClasses.append(classes[dataIndex])
				dataIndex += 1

			# Get the values in newClasses
			classValues = []
			for aclass in newClasses:
				if classValues.count(aclass) == 0:
					classValues.append(aclass)

			classCounts = np.zeros(len(classValues))
			classIndex = 0
			for classValue in classValues:
				for aclass in newClasses:
					if aclass == classValue:
						classCounts[classIndex] += 1
				classIndex += 1

			for classIndex in range(len(classValues)):
				entropy[valueIndex] += self.calc_entropy(
					float(classCounts[classIndex]) / np.sum(classCounts))
				gini[valueIndex] += (
					float(classCounts[classIndex]) / np.sum(classCounts))**2

			# Computes both the Gini gain and the entropy
			gain = gain + float(
				featureCounts[valueIndex]) / nData * entropy[valueIndex]
			ggain = ggain + float(
				featureCounts[valueIndex]) / nData * gini[valueIndex]
			valueIndex += 1
		return gain, 1 - ggain

## Testing decision tree

In [ ]:
data = [[0, 0], [1, 0], [0, 1], [1, 1]]
classes = [0, 1, 1, 1]
names = ['x1', 'x2']
tree = dtree().make_tree(data, classes, names)
tree

{'x1': {0: {'x2': {0: 0, 1: 1}}, 1: 1}}

Illistrate the resulting tree


Question #4



--- The implementation of the algorithm using Gini Impurity was already completed, thus I will explain what Gini Impurity does



*   Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset
*   Gini impurity is basically the probability we cassify a certain datapoint incorretly using the class distribution of the dataset to randomly choose the datapoints class. 
*   Gini impurity is used to determine the quality of a decision tree split by weighting the impurity of each branch by how many elements it has
*   Gini gain is the amount of impurity that was "removed" with the split
*   When training a decision tree the best split is chosen by maximizing the gini gain which is calculated by subtracting the weighted impurities of the branches from the original impurity


In [ ]:
# First Tree
#dtree.printTree(tree=tree, name=names[0])
print(tree)
# Second Tree
#dtree.printTree(tree=tree, name=names[1])

{'x1': {0: {'x2': {0: 0, 1: 1}}, 1: 1}}


## Random forest class

In [ ]:
class randomforest:
    """The random forest algorithm based on the decision tree of Chapter 6"""

    def __init__(self):
        """ Constructor """
        self.tree = dtree()

    def rf(self,
           data,
           targets,
           features,
           nTrees,
           nSamples,
           nFeatures,
           maxlevel=5):

        nPoints = np.shape(data)[0]
        nDim = np.shape(data)[1]
        self.nSamples = nSamples
        self.nTrees = nTrees

        classifiers = []

        for i in range(nTrees):
            print(i)
            # Compute bootstrap samples
            samplePoints = np.random.randint(0, nPoints, (nPoints, nSamples))

            for j in range(nSamples):
                sample = []
                sampleTarget = []
                for k in range(nPoints):
                    sample.append(data[samplePoints[k, j], :])
                    sampleTarget.append(targets[samplePoints[k, j]])
            # Train classifiers
            classifiers.append(
                self.tree.make_tree(
                    sample, sampleTarget, features, maxlevel,
                    forest=nFeatures))
        return classifiers

    def rfclass(self, classifiers, data):

        decision = []
        # Majority voting
        for j in range(len(data)):
            outputs = []
            #print data[j]
            for i in range(self.nTrees):
                out = self.tree.classify(classifiers[i], data[j, :])
                if out is not None:
                    outputs.append(out)
            # List the possible outputs
            out = []
            for each in outputs:
                if out.count(each) == 0:
                    out.append(each)
            frequency = np.zeros(len(out))

            index = 0
            if len(out) > 0:
                for each in out:
                    frequency[index] = outputs.count(each)
                    index += 1
                decision.append(out[frequency.argmax()])
            else:
                decision.append(None)
        return decision

## Testing random forest

In [ ]:
# import data
breast = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data", header=None)
print(breast.shape)
breast.head(10)

(569, 32)


0  1      2      3       4   ...      27      28      29      30       31
0    842302  M  17.99  10.38  122.80  ...  0.6656  0.7119  0.2654  0.4601  0.11890
1    842517  M  20.57  17.77  132.90  ...  0.1866  0.2416  0.1860  0.2750  0.08902
2  84300903  M  19.69  21.25  130.00  ...  0.4245  0.4504  0.2430  0.3613  0.08758
3  84348301  M  11.42  20.38   77.58  ...  0.8663  0.6869  0.2575  0.6638  0.17300
4  84358402  M  20.29  14.34  135.10  ...  0.2050  0.4000  0.1625  0.2364  0.07678
5    843786  M  12.45  15.70   82.57  ...  0.5249  0.5355  0.1741  0.3985  0.12440
6    844359  M  18.25  19.98  119.60  ...  0.2576  0.3784  0.1932  0.3063  0.08368
7  84458202  M  13.71  20.83   90.20  ...  0.3682  0.2678  0.1556  0.3196  0.11510
8    844981  M  13.00  21.82   87.50  ...  0.5401  0.5390  0.2060  0.4378  0.10720
9  84501001  M  12.46  24.04   83.97  ...  1.0580  1.1050  0.2210  0.4366  0.20750

[10 rows x 32 columns]

In [ ]:
X = breast.drop([0,1], axis=1).values
y = breast[1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
rf = randomforest()
rf_model = rf.rf(X_train, y_train, list(range(1, 31)), 20, 500, 10, maxlevel=3)
out = rf.rfclass(rf_model, X_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Illistrating the resulting tree
Question # 6

In [ ]:
# changed numtrees to 250 and number of features to 10
randFor = randomforest()
randFor_Mod = randFor.rf(X_train, y_train, list(range(1, 31)), 10, 250, 10, maxlevel=3)
out = randFor.rfclass(randFor_Mod, X_test)

0
1
2
3
4
5
6
7
8
9
